In [1]:
import cv2
import torch
from ultralytics import YOLO

from common.yolo.yolo_results_aux import results_to_pose_list, pose_list_to_numpy, sort_results_to_pose_list
from common.yolo.nms import nms_for_yolo_pose
from common.tracker.SORT import SortTracker
from common.yolo.visualization import draw_sort_bboxes, draw_skeletons_without_bboxes, draw_bboxes_with_names

# Load the model and video

In [2]:
# Load the YOLOv8n-Pose model
model = YOLO("/opt/models/yolo/yolo11m-pose.pt")

# Test whether the GPU device is available
DEVICE = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'

# Move the model to the device
model.to(DEVICE)

YOLO(
  (model): PoseModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(192, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(256, eps=0.001, momentum=0.03, affine=True, track_runni

# Test the raw output of SortTracker

In [3]:
# Open the video file
cap = cv2.VideoCapture("/opt/videos/raining_street_02.mp4")
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Define the screen and window dimensions
window_width = 1024  # Window width as specified in cv2.resize()
window_height = 800  # Window height as specified in cv2.resize()

# Create the SORT tracker
tracker = SortTracker(max_age=10, min_hits=1, iou_threshold=0.5, max_objects=100)

# Set up the display window at (0, 0) position in the top-left corner
cv2.namedWindow("Frame", cv2.WINDOW_NORMAL)

# Move window to the top-left corner
cv2.moveWindow("Frame", 0, 0)

# Set the window size to the specified width and height
cv2.resizeWindow("Frame", window_width, window_height)

# Process the video
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame
    frame = cv2.resize(frame, (460, 460))

    # Perform object detection
    results = model(frame, verbose=False)[0] # Get the first result

    # Use a list to store the detected boxes
    dets_data = []

    # Process the pose
    poses = results_to_pose_list(results)

    # Draw the skeletons without the bounding boxes
    draw_skeletons_without_bboxes(frame, poses, show_names=False)

    # Apply NMS to the poses
    poses = nms_for_yolo_pose(poses)

    # Convert the list of poses to a numpy array
    poses_array = pose_list_to_numpy(poses)

    # Perform object tracking
    tracked_objects = tracker.update(poses_array)

    # Draw the bboxes without labels
    draw_sort_bboxes(frame, tracked_objects)

    # Display the frame
    frame = cv2.resize(frame, (window_width, window_height))
    cv2.imshow("Frame", frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()

# Test the processed output of SortTracker

In [4]:
# Open the video file
cap = cv2.VideoCapture("/opt/videos/raining_street_02.mp4")
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Define the screen and window dimensions
window_width = 1024  # Window width as specified in cv2.resize()
window_height = 800  # Window height as specified in cv2.resize()

# Create the SORT tracker
tracker = SortTracker(max_age=10, min_hits=1, iou_threshold=0.5, max_objects=100)

# Set up the display window at (0, 0) position in the top-left corner
cv2.namedWindow("Frame", cv2.WINDOW_NORMAL)

# Move window to the top-left corner
cv2.moveWindow("Frame", 0, 0)

# Set the window size to the specified width and height
cv2.resizeWindow("Frame", window_width, window_height)

# Process the video
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame
    frame = cv2.resize(frame, (460, 460))

    # Perform object detection
    results = model(frame, verbose=False)[0] # Get the first result

    # Use a list to store the detected boxes
    dets_data = []

    # Process the pose
    poses = results_to_pose_list(results)

    # Draw the skeletons without the bounding boxes
    draw_skeletons_without_bboxes(frame, poses, show_names=False)

    # Apply NMS to the poses
    poses = nms_for_yolo_pose(poses)

    # Convert the list of poses to a numpy array
    poses_array = pose_list_to_numpy(poses)

    # Perform object tracking
    tracked_objects = tracker.update(poses_array)

    # Process the tracked objects
    tracked_poses, ids = sort_results_to_pose_list(tracked_objects)

    # Draw the bboxes without labels
    draw_bboxes_with_names(frame, tracked_poses, ids)

    # Display the frame
    frame = cv2.resize(frame, (window_width, window_height))
    cv2.imshow("Frame", frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()